**ÉTAPE 0** : préparation des données

In [1]:
import process 
import pandas as pd
import numpy as np

# Nail path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv'
# Jerome path : 'C:\Users\33640\OneDrive\Documents\GitHub\Portfolio_clustering_project\Data\DATA_Statapp.csv'
# Mohamed path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv'
df = pd.read_csv('/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv')

# Apply conversion function to 'open' and 'close' columns
df['open'] = df['open'].apply(process.safe_literal_eval)
df['close'] = df['close'].apply(process.safe_literal_eval)

# Calculate returns for each line
df['return'] = df.apply(lambda row: [(close - open) / open for open, close in zip(row['open'], row['close'])], axis=1)

new_df = df[['ticker', 'return']] # create a new data frame with the column ticker and return 

# Créons le DataFrame à partir des listes dans 'return'
# On suppose ici que 'new_df' est déjà défini et contient la colonne 'return'

# Convertir chaque liste dans la colonne 'return' en plusieurs colonnes dans le nouveau DataFrame
returns_df = pd.DataFrame(new_df['return'].tolist())

# Ajouter la colonne 'ticker' du 'new_df' au début de 'returns_df'
returns_df.insert(0, 'ticker', new_df['ticker'])

# Renommer les colonnes pour refléter qu'elles sont des rendements
returns_df.columns = ['ticker'] + [f'return_{i}' for i in range(len(returns_df.columns) - 1)]

df_cleaned = process.remove_rows_with_nan(returns_df)
df_cleaned.reset_index(drop=True, inplace=True)

process.check_nan_inf(df_cleaned)

df_cleaned.set_index('ticker', inplace=True)

There are no NaN values in the dataframe


**ÉTAPE 1** : Phase d'entraînement

1. Obtention de la matrice de corrélation des actifs sur une fenêtre arrière de 30 jours (1 mois)

In [2]:
number_of_repetitions = 40
lookback_window = 365
number_of_clusters = 20

consolidated_W = process.consolidated_W(lookback_window=lookback_window, df_cleaned=df_cleaned, number_of_clusters=number_of_clusters, number_of_repetitions=number_of_repetitions)

/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Code/process.py:48: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  A_pos = mat.applymap(lambda x: x if x >= 0 else 0)
/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Code/process.py:49: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  A_neg = mat.applymap(lambda x: abs(x) if x < 0 else 0)
/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Code/process.py:48: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  A_pos = mat.applymap(lambda x: x if x >= 0 else 0)
/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project

In [8]:
number_of_repetitions = 40
evaluation_window= 365
number_of_clusters = 20
training_window = 30

evaluation_set = df_cleaned.iloc[:, training_window+1:training_window+1+evaluation_window]

In [9]:
evaluation_set

,return_31,return_32,return_33,return_34,return_35,return_36,return_37,return_38,return_39,return_40,...,return_386,return_387,return_388,return_389,return_390,return_391,return_392,return_393,return_394,return_395
ticker,,,,,,,,,,,,,,,,,,,,,
AA,0.000848,0.039696,-0.017945,-0.028476,-0.025685,-0.025350,0.008079,0.020646,-0.035211,0.019178,...,-0.030073,0.001017,0.066493,-0.009780,-0.012808,-0.023457,-0.047826,0.051413,0.031030,-0.019314
ABM,0.010870,0.063158,-0.019900,-0.017766,0.025907,-0.025381,0.000000,0.018135,0.017677,0.000000,...,0.019257,0.029810,-0.006614,-0.005572,0.013333,-0.014721,-0.017135,0.002201,-0.014548,0.007003
ABT,-0.001799,-0.019678,-0.040441,-0.001869,-0.007722,-0.015504,0.005882,-0.011194,-0.003774,0.020755,...,-0.007382,0.003000,0.028942,0.000194,0.006740,-0.012865,0.012039,0.007828,0.009737,-0.001163
ADI,0.026263,0.094083,-0.037234,0.056526,0.027403,0.068295,0.002396,-0.032767,0.001592,0.015723,...,-0.064449,0.040247,-0.058795,0.038272,0.053593,-0.049480,0.024586,0.033103,0.125175,-0.026512
ADM,0.016854,0.005495,0.005495,0.005495,-0.027473,-0.005650,-0.028409,-0.011696,-0.047337,0.037736,...,-0.026148,-0.005764,-0.000726,0.002888,-0.007168,-0.043321,-0.011329,0.013846,-0.008359,-0.005356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XLY,-0.032407,-0.035629,-0.009288,-0.011097,-0.012963,-0.023960,0.003876,0.018987,0.009828,-0.012158,...,0.001743,0.009431,0.005913,0.000687,0.003097,-0.017514,-0.013986,0.028255,0.004538,-0.001739
XOM,0.008109,-0.014241,-0.029911,-0.010656,0.006745,0.001720,-0.020672,0.035211,0.024660,0.009121,...,-0.001857,-0.010670,-0.002959,-0.002778,0.004867,-0.022989,-0.035546,0.017633,0.034383,0.004230
XRX,-0.005747,-0.037464,0.005988,0.000000,-0.017857,-0.015106,-0.006173,0.027864,0.077399,-0.028169,...,-0.012761,0.003525,0.009524,-0.034118,0.001218,-0.014423,-0.013580,-0.047679,0.040312,0.000000


In [4]:

portfolio_return = process.portfolio_annualized_returns(evaluation_window=evaluation_window, df_cleaned=df_cleaned, training_window=L, final_weights=W)

UnboundLocalError: cannot access local variable 'portfolio_return' where it is not associated with a value

In [12]:
portfolio_return = (portfolio_return + 1)**(250/L) - 1

In [13]:
risk_free_rate = 0.04
SR, PNL = process.Sharpe_and_PnL(portfolio_daily_returns=portfolio_return, risk_free_rate=risk_free_rate)

In [14]:
import plotly.graph_objects as go


process.plot_PnL(SR=SR, PNL=PNL)

In [80]:

# Création du tableau synthétique
data = {
    'Rendement Moyen': [Rp],
    'Écart Type': [sigma_p],
    'Taux sans Risque': [Rf],
    'Sharpe Ratio': [SR],
    'PNL Finale': [PNL[-1]]
}

table = pd.DataFrame(data)

# Affichage du tableau
print(table)

   Rendement Moyen  Écart Type  Taux sans Risque  Sharpe Ratio  PNL Finale
0         0.016567    0.156385              0.02     -0.021955    0.496999
